# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook.  Suppose you want to predict a continuous output (regression) from a stack of continuous inputs.  In this example, the output is impervious surface area from [NLCD](https://www.mrlc.gov/data) and the input is a Landsat 8 composite.  The model is a [fully convolutional neural network (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597). This notebook shows:

1.   Exporting training/testing patches from Earth Engine, suitable for training an FCNN model.
2.   Preprocessing.
3.   Training and validating an FCNN model.
4.   Making predictions with the trained model and importing them to Earth Engine.

# Variables

Declare the variables that will be in use throughout the notebook.

In [1]:
PACKAGE_PATH = 'Water_classification_package'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 4
drwxr-xr-x 1 root root 4096 Jul  6 13:22 sample_data
total 0
-rw-r--r-- 1 root root 0 Jul  9 07:06 __init__.py


In [2]:
%%writefile {PACKAGE_PATH}/metrics.py

from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

# Acc = TP + TN / (TP + TN + FP + FN)
# possible_pos = TP + FN
# predicted_pos = TP + FP
# Missing TN
# TN = total - possible_pos - predicted_pos + TP
# TP + TN + FP + FN = possible_pos + predicted_pos - TP + TN

def custom_accuracy(y_true, y_pred):
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + possible_positives + predicted_positives
  
    # total_data = tf.cast(total_data, tf.float32|tf.int32)
    # true_positives = tf.cast(true_positives, tf.float32|tf.int32)
    # possible_positives = tf.cast(possible_positives, tf.float32|tf.int32)
    # predicted_positives = tf.cast(predicted_positives, tf.float32|tf.int32)
    # print(K.int_shape(y_true), K.int_shape(y_pred))
    # print(K.int_shape(y_pred)[0], K.int_shape(y_pred)[1], K.int_shape(y_pred)[2])
    # print(total_data)
    # print(possible_positives)
    # (true_positives) / (total_data + K.epsilon())
    return (true_positives + true_negatives) / (total_data + K.epsilon())

Writing Water_classification_package/metrics.py


In [3]:

project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

Updated property [core/project].


In [4]:
%%writefile {PACKAGE_PATH}/config.py

# Tensorflow setup.
import tensorflow as tf
from . import metrics

# INSERT YOUR BUCKET HERE:
BUCKET = 'geebucketwater'


# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'Cnn_S1_DEM'
TRAINING_BASE = 'training_patches_S1_DEM'
EVAL_BASE = 'eval_patches_S1_DEM'
TEST_BASE_1 = 'test_patches_S1_DEM_1'
TEST_BASE_2 = 'test_patches_S1_DEM_2'

# Specify inputs (Landsat bands) to the model and the response variable.
BANDS = ['VV', "VH", "angle", "slope", "aspect", "elevation"]
# thermalBands = ['B10', 'B11']
RESPONSE = 'water'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 1
BUFFER_SIZE = 2000
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['AUC', metrics.f1, metrics.custom_accuracy]
# METRICS = ['RootMeanSquaredError']

Writing Water_classification_package/config.py


In [5]:
%%writefile {PACKAGE_PATH}/preprocessing.py

from . import config
import tensorflow as tf

def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, config.FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in config.FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(config.BANDS)], tf.reshape(tf.one_hot(tf.cast(stacked[:,:,len(config.BANDS):], tf.int32), depth=2),[256,256,2])


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + config.BUCKET + '/' + config.FOLDER + '/' + config.TRAINING_BASE + '*'
	print(glob)
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(config.BUFFER_SIZE).batch(config.BATCH_SIZE).repeat()
	return dataset


# print(iter(training.take(1)).next())

def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + config.BUCKET + '/' + config.FOLDER + '/' + config.EVAL_BASE + '*'
	print(glob)
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

# print(iter(evaluation.take(1)).next())

def get_test_dataset(folder):
  """Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
  glob = 'gs://' + config.BUCKET + '/' + config.FOLDER + '/' + folder + '*'
  print(glob)
  dataset = get_dataset(glob)
  dataset = dataset.batch(1).repeat()
  return dataset



Writing Water_classification_package/preprocessing.py


In [6]:
%%writefile {PACKAGE_PATH}/visualizer.py
import folium
def mapping(image, bandName, min, max, name):
  mapid = image.getMapId({'bands': [bandName], 'min': min, "max": max})
  map = folium.Map(location=[38., -122.5])
  folium.TileLayer(
      tiles=mapid['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      overlay=True,
      name=name,
    ).add_to(map)

  map.add_child(folium.LayerControl())
  return map

Writing Water_classification_package/visualizer.py


# Setup software libraries

Authenticate and import as necessary.

In [7]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

from Water_classification_package import config, visualizer, metrics, preprocessing

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=RndpjXhoOGyUixUjcY4xNBn2k7RN_r7moxuqAbUlXM8&tc=tM9F8c8WRs9rIa1I4EysGYcToSumVXQpNGRqjBnC1M8&cc=8nvD4rGTkGnq1ZCBd5a0Vt1ZXnXuNDJ4Sfl5UqWzjk8

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qjAS_dQOlDWpQHHmv3CZxHoLBVU9XL6PheGyGhtLKOKzj4wWLQbPeI

Successfully saved authorization token.
2.8.2


# Imagery

Gather and setup the imagery to use for inputs (predictors).  This is a three-year, cloud-free, Landsat 8 composite.  Display it in the notebook for a sanity check.

In [8]:
# Use Landsat 8 surface reflectance data.
S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filterDate('2018-01-01','2018-02-01') \
        # .filterDate('2017-01-01','2018-02-01')

dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)
image = ee.Image.cat([
  S1.median(),
  dem,
  slope,
  aspect
]).float()

waterdata = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filterDate('2018-01-01', '2018-02-01').median()
watermask = waterdata.select("water")
mask = watermask.gt(0)
maskedComposite = waterdata.updateMask(mask).subtract(1)

Stack the 2D images (Landsat composite and NLCD impervious surface) to create a single image from which samples can be taken.  Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [ ]:
featureStack = ee.Image.cat([
  image.select(config.BANDS),
  maskedComposite.select(config.RESPONSE)
]).float()

list = ee.List.repeat(1, config.KERNEL_SIZE)
lists = ee.List.repeat(list, config.KERNEL_SIZE)
kernel = ee.Kernel.fixed(config.KERNEL_SIZE, config.KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)
print(arrays.getInfo())

{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}, {'id': 'aspect', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}, {'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimens

Use some pre-made geometries to sample the stack in strategic locations.  Specifically, these are hand-made polygons in which to take the 256x256 samples.  Display the sampling polygons on a map, red for training polygons, blue for evaluation.

In [ ]:
import folium
# trainingPolys = ee.FeatureCollection('projects/google/DemoTrainingGeometries')
trainingPolys = ee.FeatureCollection('users/mewchayutaphong/thailandTraining')
# evalPolys = ee.FeatureCollection('projects/google/DemoEvalGeometries')
first = ee.Geometry.BBox(101.78381817548382, 14.052178100305664, 102.27820294110882, 14.361037359593043);
second = ee.Geometry.BBox(102.16833965985882, 16.426385350573945, 102.83850567548382, 16.921030330473783);
evalPolys = ee.FeatureCollection(first).merge(second)
testPolys1 = ee.FeatureCollection(ee.Geometry.BBox(106.42682423644103, 15.001360239504, 106.8014457430141, 15.59647593973863))
testPolys2 = ee.FeatureCollection(ee.Geometry.BBox(70.3090276140532, 31.351437257990685, 71.5724553484282, 32.32209892418571))

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2).paint(testPolys1, 3).paint(testPolys2, 4)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 3, 'palette': ['red', 'blue', "green", "purple"]})
map = folium.Map(location=[16.426385350573945, 102.16833965985882], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Sampling

The mapped data look reasonable so take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data for the FCNN, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers were determined experimentally.
n = 200 # Number of shards in each polygon.
N = 2000 # Total sample size in each polygon.

# Export all the training data (in many pieces), ith one task 
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30,
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
    geomSample = geomSample.merge(sample)

  desc = config.TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = config.BUCKET,
    fileNamePrefix = config.FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = config.BANDS + [config.RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30,
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
    geomSample = geomSample.merge(sample)

  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = BUCKET,
    fileNamePrefix = config.FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = config.BANDS + [config.RESPONSE]
  )
  task.start()

In [ ]:
testPolys1List = testPolys1.toList(testPolys1.size())
testPolys2List = testPolys2.toList(testPolys2.size())


n = 200 # Number of shards in each polygon.
N = 2000 # Total sample size in each polygon.

# Export all the test data.
for g in range(testPolys1.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(testPolys1List.get(g)).geometry(), 
      scale = 30,
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
    geomSample = geomSample.merge(sample)

  desc = config.TEST_BASE_1 + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = config.BUCKET,
    fileNamePrefix = config.FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = config.BANDS + [config.RESPONSE]
  )
  task.start()

# Export all the test2 data.
for g in range(testPolys2.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(testPolys2List.get(g)).geometry(), 
      scale = 30,
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
    geomSample = geomSample.merge(sample)

  desc = config.TEST_BASE_2 + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = config.BUCKET,
    fileNamePrefix = config.FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = config.BANDS + [config.RESPONSE]
  )
  task.start()

In [ ]:
# Print all tasks.
from pprint import pprint
# pprint(ee.batch.Task.list())
for i in range(3):
  pprint(ee.batch.Task.list()[i])

<Task DZNT7JENLFVDQXLEGHEKBIIH EXPORT_FEATURES: test_patches_S1_DEM_2_g0 (COMPLETED)>
<Task ZCGNLXKJ5AKTDRKYU3V5W7DL EXPORT_FEATURES: test_patches_S1_DEM_1_g0 (COMPLETED)>
<Task N4DBOSE3EPTELVECZ3YM4DEP EXPORT_FEATURES: eval_patches_S1_DEM_g1 (COMPLETED)>


In [ ]:
!earthengine task cancel all

Canceling task "ZR7VW6COKRPDX66PDFMAWXF5"
Canceling task "X62KTRH4SZZLJTGPKZSKRDM4"
Canceling task "LKASP242HNQOWMYRSPFZDG7M"
Canceling task "OVXYIUXWHIOTFNDA7C6UCRNN"
Canceling task "JMJMK2MM7IBKUKGTSO5KLHDC"
Canceling task "QCCKC6NPOG2EUZLDI2FJ73I3"


# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [9]:
training = preprocessing.get_training_dataset()
evaluation = preprocessing.get_eval_dataset()
test_1 = preprocessing.get_test_dataset(config.TEST_BASE_1)
test_2 = preprocessing.get_test_dataset(config.TEST_BASE_2)

gs://geebucketwater/Cnn_S1_DEM/training_patches_S1_DEM*
gs://geebucketwater/Cnn_S1_DEM/eval_patches_S1_DEM*
gs://geebucketwater/Cnn_S1_DEM/test_patches_S1_DEM_1*
gs://geebucketwater/Cnn_S1_DEM/test_patches_S1_DEM_2*


# Model

Here we use the Keras implementation of the U-Net model.  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.  Since impervious surface fraction is constrained to [0,1], with many values close to zero or one, a saturating activation function is suitable here.

In [12]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
# from tensorflow.python.keras.layers.experimental.preprocessing.Normalization import BatchNormalization


def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def EncoderMiniBlock(inputs, n_filters=32, dropout_prob=0.3, max_pooling=True):
    conv = layers.Conv2D(n_filters, 
                  3,  # filter size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(inputs)
    conv = layers.Conv2D(n_filters, 
                  3,  # filter size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(conv)
  
    conv = layers.BatchNormalization()(conv, training=False)
    if dropout_prob > 0:     
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)
    if max_pooling:
        next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv)    
    else:
        next_layer = conv
    skip_connection = conv    
    return next_layer, skip_connection

def DecoderMiniBlock(prev_layer_input, skip_layer_input, n_filters=32):
    up = layers.Conv2DTranspose(
                 n_filters,
                 (3,3),
                 strides=(2,2),
                 padding='same')(prev_layer_input)
    merge = layers.concatenate([up, skip_layer_input], axis=3)
    conv = layers.Conv2D(n_filters, 
                 3,  
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(merge)
    conv = layers.Conv2D(n_filters,
                 3, 
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(conv)
    return conv


In [13]:
def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = EncoderMiniBlock(inputs, 32) # 128
	encoder1_pool, encoder1 = EncoderMiniBlock(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = EncoderMiniBlock(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = EncoderMiniBlock(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = EncoderMiniBlock(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = DecoderMiniBlock(center, encoder4, 512) # 16
	decoder3 = DecoderMiniBlock(decoder4, encoder3, 256) # 32
	decoder2 = DecoderMiniBlock(decoder3, encoder2, 128) # 64
	decoder1 = DecoderMiniBlock(decoder2, encoder1, 64) # 128
	decoder0 = DecoderMiniBlock(decoder1, encoder0, 32) # 256
	# outputs = layers.Conv2D(3, (1, 1), activation='softmax')(decoder0)
	# outputs = layers.Conv2D(3, (1, 1), activation='softmax')(decoder0)
	outputs = layers.Dense(2, activation=tf.nn.softmax)(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

# Landsat Training from Scratch

In [ ]:
@tf.function
def train_step(x, y):
    print("hi")
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    # print(float(train_acc_metric.result()))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, tf.math.argmax(val_logits))

# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

Train_values = []
Val_values = []

# Prepare the metrics.
train_acc_metric = tf.keras.metrics.AUC()
val_acc_metric = tf.keras.metrics.AUC()
train_acc_metric.reset_states()

model = get_model()
import time
epochs = 1
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(evaluation):
        loss_value = train_step(x_batch_train, y_batch_train)
        print(step)

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),),)
    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in evaluation:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()

    Train_values.append(float(train_acc))
    Val_values.append(float(val_acc))
    print("Training acc over epoch: %.4f" % (float(train_acc),), "Validation acc: %.4f" % (float(val_acc),), "Time taken: %.2fs" % (time.time() - start_time))
    # print("Validation acc: %.4f" % (float(val_acc),))
    # print("Time taken: %.2fs" % (time.time() - start_time))

In [ ]:
print("hi")

In [ ]:
model.save("/")
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model.load_weights('./checkpoints/my_checkpoint')

# Custom Keras training

In [14]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers

class CustomModel(tf.keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        # print(x.numpy())

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

# Define model here
inputs = layers.Input(shape=[None, None, len(config.BANDS)]) # 256
encoder0_pool, encoder0 = EncoderMiniBlock(inputs, 32) # 128
encoder1_pool, encoder1 = EncoderMiniBlock(encoder0_pool, 64) # 64
encoder2_pool, encoder2 = EncoderMiniBlock(encoder1_pool, 128) # 32
encoder3_pool, encoder3 = EncoderMiniBlock(encoder2_pool, 256) # 16
encoder4_pool, encoder4 = EncoderMiniBlock(encoder3_pool, 512) # 8
center = conv_block(encoder4_pool, 1024) # center
decoder4 = DecoderMiniBlock(center, encoder4, 512) # 16
decoder3 = DecoderMiniBlock(decoder4, encoder3, 256) # 32
decoder2 = DecoderMiniBlock(decoder3, encoder2, 128) # 64
decoder1 = DecoderMiniBlock(decoder2, encoder1, 64) # 128
decoder0 = DecoderMiniBlock(decoder1, encoder0, 32) # 256
# outputs = layers.Conv2D(3, (1, 1), activation='softmax')(decoder0)
# outputs = layers.Conv2D(3, (1, 1), activation='softmax')(decoder0)
outputs = layers.Dense(2, activation=tf.nn.softmax)(decoder0)

model_custom = CustomModel(inputs, outputs)

model_custom.compile(
  optimizer=optimizers.get(config.OPTIMIZER), 
  loss=losses.get(config.LOSS),
  metrics=[config.METRICS])

In [ ]:
for EPOCHS in range(1,4):
  EPO = [i for i in range(1, EPOCHS + 1)]
  model_custom = CustomModel(inputs, outputs)
  model_custom.compile(
  optimizer=optimizers.get(config.OPTIMIZER), 
  loss=losses.get(config.LOSS),
  metrics=[config.METRICS])
  history = model_custom.fit(
      x=training,
      epochs=EPOCHS,
      steps_per_epoch=int(2000*3 / config.BATCH_SIZE),
      validation_data=evaluation,
      validation_steps=2000*2)

  Model_name = "S1_DEM" + "_EPOCHS_" + str(config.EPOCHS)
  MODEL_DIR = 'gs://' + config.BUCKET + "/" + config.FOLDER + "/Models/" + Model_name
  model_custom.save(config.MODEL_DIR, save_format='tf')
  hist_keys = [*history.history]

  import matplotlib.pyplot as plt
  fig, axes = plt.subplots(1,2, figsize=(12,5))
  axes[0].plot(EPO, history.history[hist_keys[0]])
  axes[0].plot(EPO, history.history[hist_keys[2]])
  axes[0].legend(['Loss', 'Val_loss'])
  axes[1].plot(EPO, history.history[hist_keys[1]])
  axes[1].plot(EPO, history.history[hist_keys[3]])
  axes[1].legend(['AUC', 'Val_AUC'])
  fig.savefig(f'epoch{EPOCHS}.png')

In [ ]:
MODEL_DIR_1 = 'gs://' + config.BUCKET + "/" + config.FOLDER + "/Models/S1CNNmodel_EPOCHS_" + str(1)
MODEL_DIR_2 = 'gs://' + config.BUCKET + "/" + config.FOLDER + "/Models/S1CNNmodel_EPOCHS_" + str(2)
MODEL_DIR_3 = 'gs://' + config.BUCKET + "/" + config.FOLDER + "/Models/S1CNNmodel_EPOCHS_" + str(3)

print("loading...", MODEL_DIR_1)
model_custom_1EP = tf.keras.models.load_model(MODEL_DIR_1)
print("loading...", MODEL_DIR_1)
model_custom_2EP = tf.keras.models.load_model(MODEL_DIR_2)
print("loading...", MODEL_DIR_1)
model_custom_3EP = tf.keras.models.load_model(MODEL_DIR_3)

loading...
loading...
loading...


# Automatic Fit keras Training

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [ ]:
model = get_model()

model.fit(
    x=evaluation,
    epochs=EPOCHS,
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE),
    validation_data=evaluation,
    validation_steps=EVAL_SIZE)

Note that the notebook VM is sometimes not heavy-duty enough to get through a whole training job, especially if you have a large buffer size or a large number of epochs.  You can still use this notebook for training, but may need to set up an alternative VM ([learn more](https://research.google.com/colaboratory/local-runtimes.html)) for production use.  Alternatively, you can package your code for running large training jobs on Google's AI Platform [as described here](https://cloud.google.com/ml-engine/docs/tensorflow/trainer-considerations).  The following code loads a pre-trained model, which you can use for predictions right away.

In [ ]:
# # Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
# MODEL_DIR = 'gs://ee-docs-demos/fcnn-demo/trainer/model'
# m = tf.keras.models.load_model(MODEL_DIR)
# m.summary()

# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storge bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.

The following functions handle this process.  It's useful to separate the export from the predictions so that you can experiment with different models without running the export every time.

In [ ]:
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS),
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
import numpy as np
def doPrediction(out_image_base, user_folder, kernel_buffer, region, model, suffix):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = model.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
    predictionPatch = np.argmax(predictionPatch, -1)
    # print(predictionPatch)
    # print(predictionPatch.shape)
    # print("pred", tf.argmax(predictionPatch, 1))
    # print(predictionPatch[0][0])
    # print(predictionPatch[0][1])
    # print(predictionPatch[0][2])
    
#     patch[0].append(tf.argmax(predictionPatch, 1))
#     patch[1].append(predictionPatch[0][0])
#     patch[2].append(predictionPatch[0][1])
#     patch[3].append(predictionPatch[0][2])
    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base + suffix
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/mewchayutaphong' # INSERT YOUR FOLDER HERE.

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]
bj_kernel_buffer = [128, 128]
# Base file name to use for TFRecord files and assets.
bj_image_base = 'FCNN_demo_beijing_384_S1'
# Beijing
bj_region = ee.Geometry.Polygon(
        [[[115.9662455210937, 40.121362012835235],
          [115.9662455210937, 39.64293313749715],
          [117.01818643906245, 39.64293313749715],
          [117.01818643906245, 40.121362012835235]]], None, False)

th_image_base = "Thai_water_with_S1"
th_region = ee.Geometry.BBox(106.42682423644103, 15.001360239504, 106.8014457430141, 15.59647593973863)


np_image_base = "Nepal_water_with_S1"
np_region = ee.Geometry.BBox(70.3090276140532, 31.351437257990685, 71.5724553484282, 32.32209892418571)


In [ ]:
# Run the export.
# doExport(bj_image_base, bj_kernel_buffer, bj_region)
doExport(np_image_base, kernel_buffer, np_region)

In [ ]:
doExport(th_image_base, kernel_buffer, th_region)

Running image export to Cloud Storage...
Image export completed.


In [ ]:
doPrediction(th_image_base, user_folder, kernel_buffer, th_region, model_custom_1EP, "_epochs_1_")
doPrediction(th_image_base, user_folder, kernel_buffer, th_region, model_custom_2EP, "_epochs_2_")
doPrediction(th_image_base, user_folder, kernel_buffer, th_region, model_custom_3EP, "_epochs_3_")

Looking for TFRecord files...
['gs://geebucketwater/fcnn-demo_S1/Thai_water_with_S1.tfrecord.gz']
gs://geebucketwater/fcnn-demo_S1/Thai_water_with_S1.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 5,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            106.42664564466256,
                                            0.0,
                                            -0.00026949458523585647,
                                            15.596729625939957]},
                'crs': 'EPSG:4326'},
 'totalPatches': 40}
Running predictions...
40/40 [==============================] - 9s 150ms/step
Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writing patch 13..

In [ ]:
# Run the prediction.
# doPrediction(bj_image_base, user_folder, bj_kernel_buffer, bj_region)
doPrediction(np_image_base, user_folder, kernel_buffer, np_region, model_custom_1EP, "_epochs_1")
doPrediction(np_image_base, user_folder, kernel_buffer, np_region, model_custom_2EP, "_epochs_2")
doPrediction(np_image_base, user_folder, kernel_buffer, np_region, model_custom_3EP, "_epochs_3")

## Accuracy

In [ ]:
# from tqdm import tqdm
import tqdm.notebook as tq
import numpy as np

def MetricCalculator(model, test_data, total_steps):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  # total_steps = 2000
  test_acc_metric = tf.keras.metrics.Accuracy()
  # test_F1_metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5)
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    # print(y_pred[0][1] == 1, y_pred[0][1] == 1)
    for j in range(y_pred.shape[0]):
      if y_true[j][1] == 1 and y_pred[j][1] == 1:
        TP += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 0:
        TN += 1
      if y_true[j][1] == 1 and y_pred[j][1] == 0:
        FN += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 1:
        FP += 1
    test_acc_metric.update_state(y_true, y_pred)
    # # recall_metric.update_state(data[1], test_logits)
    # # precision_metric.update_state(data[1], test_logits)

  print(test_acc_metric.result().numpy())
  print("TP: ", TP)
  print("TN: ", TN)
  print("FP: ", FP)
  print("FN: ", FN)
  precision = TP/(TP + FP)
  recall = TP/(TP + FN)

  print("precision", precision)
  print("recall", recall)

  F1 = 2*(recall*precision)/(recall + precision)
  print(F1)

  print(TP, TN, FP, FN)
  return F1, test_acc_metric.result().numpy()



In [ ]:
MetricCalculator(model_custom_1EP, test_1, 2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

0.99617374
TP:  1516497
TN:  129053939
FP:  48497
FN:  453067
precision 0.9690113827912439
recall 0.7699658401554862
0.8580971086059416
1516497 129053939 48497 453067


(0.8580971086059416, 0.99617374)

In [ ]:
MetricCalculator(model_custom_2EP, test_1, 2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

0.9940117
TP:  1194074
TN:  129092936
FP:  9500
FN:  775490
precision 0.9921068417895368
recall 0.6062631120390096
0.7526139739273867
1194074 129092936 9500 775490


(0.7526139739273867, 0.9940117)

In [ ]:
MetricCalculator(model_custom_3EP, test_1, 2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

0.99250096
TP:  987610
TN:  129101303
FP:  1133
FN:  981954
precision 0.9988541006105732
recall 0.5014358507771263
0.6676859433452985
987610 129101303 1133 981954


(0.6676859433452985, 0.99250096)

In [ ]:
MetricCalculator(model_custom_1EP, test_2, 2000)

In [ ]:
MetricCalculator(model_custom_2EP, test_2, 2000)

In [ ]:
MetricCalculator(model_custom_3EP, test_2, 2000)

# Display the output

One the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the impervious area predictions over Beijing, China.

In [ ]:
user_folder + '/' + bj_image_base

In [ ]:
out_image = ee.Image(user_folder + '/' + bj_image_base)
mapid = out_image.getMapId({'min': 0, 'max': 2})
map = folium.Map(location=[39.898, 116.5097])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map